###  Извлечение именованных сущностей (NER) и отношений

b-person
i-person
i-person
i-person
o-person

person
no_person

Одна из самых популярных задач NLP – извлечении именованных сущностей (Named-entity recognition, NER). Задача NER – выделить спаны сущностей в тексте (спан – непрерывный фрагмент текста). Допустим, есть новостной текст, и мы хотим выделить в нем сущности (некоторый заранее зафиксированный набор — например, персоны, локации, организации, даты и так далее). Задача NER – понять, что участок текста “1 января 1997 года” является датой, “Кофи Аннан” – персоной, а “ООН” – организацией.

##### Зачем?
1.  Cтруктуризация неструктурированных данных. Пусть у вас есть какой-то текст (или набор текстов), и данные из него нужно ввести в базу данных (таблицу). 
2. Шаг в сторону “понимания” текста. Это может как иметь самостоятельную ценность, так и помочь лучше решать другие задачи NLP. Без решения задачи NER тяжело представить себе решение многих задач NLP, допустим, разрешение местоименной анафоры или построение вопросно-ответных систем. 

Пример 1: Местоименная анафора позволяет нам понять, к какому элементу текста относится местоимение. Например, пусть мы хотим проанализировать текст “Прискакал Чарминг на белом коне. Принцесса выбежала ему навстречу и поцеловала его”. Если мы выделили на слове “Чарминг” сущность Персона, то машина сможет намного легче понять, что принцесса, скорее всего, поцеловала не коня, а принца Чарминга.

Пример 2: Теперь приведем пример, как выделение именованных сущностей может помочь при построении вопросно-ответных систем. Если задать в вашем любимом поисковике вопрос «Кто играл роль Дарта Вейдера в фильме “Империя наносит ответный удар”», то с большой вероятностью вы получите верный ответ. Это делается как раз с помощью выделения именованных сущностей: выделяем сущности (фильм, роль и т. п.), понимаем, что нас спрашивают, и дальше ищем ответ в базе данных.

Для решения NER-задач существуют множество инструментов. Рассмотрим несколько из них.


#### NLTK

<img src='image/nltk.PNG'>

Для разметки NER с помощью NLTK сначала производим токенизацию слов, затем POS тэггинг. 
В качестве документа возьмем статью 'https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news, распарсим ее с помощью BeautifulSoup.

In [1]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import nltk

In [3]:
import requests
from bs4 import BeautifulSoup
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

document = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')

nltk.pos_tag(nltk.word_tokenize(document))

[('nytimes.comPlease', 'NN'),
 ('enable', 'JJ'),
 ('JS', 'NNP'),
 ('and', 'CC'),
 ('disable', 'JJ'),
 ('any', 'DT'),
 ('ad', 'NN'),
 ('blocker', 'NN')]

С помощью функции nltk.ne_chunk () мы можем распознавать именованные сущности с помощью классификатора, который добавляет метки категорий, такие как PERSON, ORGANIZATION и GPE.

In [4]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

set()

#### Spacy

Spacy значительно быстрее NLTK, так как она написана на Cython и работает с объектами.
Для NER Spacy работает как простой классификатор (неглубокая нейронная сеть с одним скрытым слоем). Объект Doc хранит последовательности токенов и все их аннотации.

<img src='image/spacy.PNG'>

In [5]:
#!pip -q install spacy
#!python -m spacy download en
#!python -m spacy download en_core_web_sm

In [6]:
#!pip install -U spacy
#!python -m spacy info
import spacy
from spacy import displacy
import en_core_web_md


#### Deeppavlov

DeepPavlov-это библиотека ИИ с открытым исходным кодом, построенная на TensorFlow и Keras. DeepPavlov предназначена для разработки чат-ботов и сложных разговорных систем, исследований в области nlp и, в частности, диалоговых систем.

DeepPavlov использует несколько более новый вариант глубокой нейронной архитектуры Flair, известный как гибридная модель Bi-LSTM-CRF.

<img src='image/deeppavlov.PNG'>

Используем предтренированную модель "ner_ontonotes_bert_mult", которая работает с разными языками, в том числе и русским. Подадим на распознование предложение на русском языке. 

In [7]:
nlp = en_core_web_md.load()
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')


C:\Users\Armik\AppData\Roaming\Python\Python37\site-packages\spacy\displacy\__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [9]:
# !python -m venv env 
# #.\env\Scripts\activate.bat
# !pip install deeppavlov
# !python -m deeppavlov install squad_bert

#!python -m deeppavlov install ner_ontonotes
import deeppavlov
from deeppavlov import configs, build_model


In [13]:
deeppavlov_ner = build_model(configs.ner.ner_ontonotes_bert, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

2023-05-03 20:52:05.557 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_torch_crf.tar.gz download because of matching hashes
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializ

[[['Нью',
   '-',
   'Йорк',
   ',',
   'США',
   ',',
   '30',
   'апреля',
   '2020',
   ',',
   '01',
   ':',
   '01',
   '—',
   'REGNUM',
   'В',
   'администрации',
   'президента',
   'США',
   'Дональда',
   'Трампа',
   'планируют',
   'пройти',
   'все',
   'этапы',
   'создания',
   'вакцины',
   'от',
   'коронавируса',
   'в',
   'ускоренном',
   'темпе',
   'и',
   'выпустить',
   '100',
   'млн',
   'доз',
   'до',
   'конца',
   '2020',
   'года',
   ',',
   'передаёт',
   'агентство',
   'Bloomberg',
   'со',
   'ссылкой',
   'на',
   'осведомлённые',
   'источники']],
 [['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-CARDINAL',
   'O',
   'B-DATE',
   'O',
   'B-TIME',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   '

### Извлечение отношений

Можно извлекать не только именнованные сущности (NER), но и отношения между словами в предложении (Relation extraction).

#### Spacy

В уже знакомой нам библеотеке Spacy используем встроенные displaCy визуализатор со style="dep" для отображения отношений.

In [14]:
doc = nlp("I think Barack Obama met founder of Facebook at occasion of a release of a new NLP algorithm.")
displacy.render(doc, style="dep") # (1)
displacy.render(doc, style="ent") # (2)

#### NLTK

Распарсим документ nltk.corpus.ieer.parsed_docs('NYT_19980315'), extract_rels позволяет нам выделять необходимые отношения, в данном случае мы ищем пары сущностей <'ORG', 'LOC'>. Также мы указали, что текст должен подходить под регулярное выражение IN.

In [15]:
import re
import nltk
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
     for rel in nltk.sem.extract_rels('ORG', 'LOC', doc,
                                      corpus='ieer', pattern = IN):
            print(nltk.sem.rtuple(rel))

[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']


#### Allennlp

Попробовать извлекать отношения онлайн можно с помощью демо Allennlp  https://demo.allennlp.org/dependency-parsing/. 
Allen NLP предлагает две модели NER с различными архитектурами: Gated Recurrent Unit (GRU) Network, bi-LSTM-CRF model.

<img src='image/allennlp.PNG'>

In [16]:
from pymorphy2 import MorphAnalyzer

In [17]:
morpher = MorphAnalyzer()

In [18]:
morpher.parse("кошка")[0]

Parse(word='кошка', tag=OpencorporaTag('NOUN,anim,femn sing,nomn'), normal_form='кошка', score=0.9375, methods_stack=((DictionaryAnalyzer(), 'кошка', 134, 0),))

In [19]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [20]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры. Свое заявление он разместил в Twitter. «Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией. Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат. 11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года). В июле аналогичное решение принял Житомирский областной совет. В начале месяца с предложением к президенту страны Петру Порошенко вернуть Бандере звание Героя Украины обратились депутаты Верховной Рады. Парламентарии уверены, что признание Бандеры национальным героем поможет в борьбе с подрывной деятельностью против Украины в информационном поле, а также остановит «распространение мифов, созданных российской пропагандой». Степан Бандера (1909-1959) был одним из лидеров Организации украинских националистов, выступающей за создание независимого государства на территориях с украиноязычным населением. В 2010 году в период президентства Виктора Ющенко Бандера был посмертно признан Героем Украины, однако впоследствии это решение было отменено судом. '
doc = Doc(text)

In [21]:
doc.segment(segmenter)
display(doc.tokens[:5])
display(doc.sents[:5])

[DocToken(stop=5, text='Посол'),
 DocToken(start=6, stop=13, text='Израиля'),
 DocToken(start=14, stop=16, text='на'),
 DocToken(start=17, stop=24, text='Украине'),
 DocToken(start=25, stop=30, text='Йоэль')]

[DocSent(stop=218, text='Посол Израиля на Украине Йоэль Лион признался, чт..., tokens=[...]),
 DocSent(start=219, stop=257, text='Свое заявление он разместил в Twitter.', tokens=[...]),
 DocSent(start=258, stop=424, text='«Я не могу понять, как прославление тех, кто непо..., tokens=[...]),
 DocSent(start=425, stop=592, text='Украина не должна забывать о преступлениях, совер..., tokens=[...]),
 DocSent(start=593, stop=798, text='11 декабря Львовский областной совет принял решен..., tokens=[...])]

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

In [22]:
# !pip install corus

In [23]:
import corus

In [44]:
from corus import load_ne5

records = load_ne5('../data/Collection5/')



In [46]:
next(records)

StopIteration: 

In [38]:
from razdel import tokenize

In [45]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 171: character maps to <undefined>

In [40]:
import pandas as pd

In [41]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [42]:
df_words['tag'].value_counts()

Series([], Name: tag, dtype: int64)

In [43]:
df_words.head(3)

,word,tag


In [ ]:
df_words.shape

In [ ]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
train_x.apply(len).max(axis=0)

In [ ]:
# char level
#train_x = train_x.apply(lambda x: ' '.join(list(x)))
#valid_x = valid_x.apply(lambda x: ' '.join(list(x)))

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    #ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [ ]:
len(vectorize_layer.get_vocabulary())

In [ ]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [ ]:
mmodel = modelNER()

In [ ]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

In [49]:
from razdel import tokenize
import json
from tqdm import notebook
from corus import load_rudrec

In [50]:
docs = []
for rec in load_rudrec("../data/rudrec_annotated.json"):
    words = []
    labels = []
    idx_ent = -1
    len_ents = len(rec.entities)
    rec_entities = sorted(rec.entities, key=lambda v: v.start)
    ent = None
    is_start = None
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        if len_ents == 0:
            words.append(token.text)
            labels.append(type_ent)
            continue
        if (idx_ent == -1) or (idx_ent + 1 < len_ents and token.start > ent.end):
            idx_ent += 1
            ent = rec_entities[idx_ent]
            is_start = True
            
        if (token.start >= ent.start) and (token.stop <= ent.end):
            type_ent = 'B-' + ent.entity_type if is_start else 'I-' + ent.entity_type
        
        words.append(token.text)
        labels.append(type_ent)
        
    docs.append([words, labels])

In [51]:
with open('rudrec.bio', 'w') as w:
    with open('rudrec_train.bio', 'w') as w1:
        with open('rudrec_valid.bio', 'w') as w2:
            for irec, rec in enumerate(docs):
                for line in map(lambda vl: '\t'.join(vl) + '\n', zip(*rec)):
                    w.write(line)
                    if irec < 3000:
                        w1.write(line)
                    else:
                        w2.write(line)
                w.write('\n')
                if irec < 3000:
                    w1.write('\n')
                else:
                    w2.write('\n')
            
            
            

In [52]:
docs[1]

[['Общее', 'впечатление', ':', 'не', 'подошел'],
 ['OUT', 'OUT', 'OUT', 'OUT', 'OUT']]

In [53]:
data, labels = list(zip(*docs))

In [54]:
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

нам	OUT
прописали	OUT
,	OUT
так	OUT
мой	OUT
ребенок	OUT
сыпью	B-ADR
покрылся	B-ADR
,	OUT
глаза	B-ADR
опухли	B-ADR
,	OUT
сверху	OUT
и	OUT
снизу	OUT
на	B-ADR
веках	B-ADR
высыпала	B-ADR
сыпь	B-ADR
,	OUT
(	OUT
8	OUT
месяцев	OUT
сыну	OUT
)	OUT
А	OUT
от	OUT
виферона	B-Drugform
такого	OUT
не	OUT
было	OUT
...	OUT
У	OUT
кого	OUT
ещё	OUT
такие	OUT
побочки	OUT
,	OUT
отзовитесь	OUT
!	OUT
1	OUT
Чем	OUT
спасались	OUT
?	OUT


In [ ]:
import spacy
import eli5